## Подготовка данных

In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
local_crs = 4326
example_data_path = "./VRP/exp_data/"

In [2]:
dodo = gpd.read_file(os.path.join(example_data_path, "Данные о доходах додо пиццы.geojson")).to_crs(local_crs)
blocks = gpd.read_parquet(os.path.join(example_data_path, "blocks_cutter_result.parquet")).to_crs(local_crs)
accessibility_matrix = pd.read_pickle(os.path.join(example_data_path, "matrix_mon_0900.pickle"))
demands = pd.read_csv(os.path.join(example_data_path, "second_scenario.csv"))

demands.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

In [3]:
blocks['centroid'] = blocks['geometry'].centroid
blocks.geometry = blocks['centroid']
blocks.drop(columns=['centroid', 'landuse'], inplace=True)


/var/folders/h8/0wmx2zx90bn60jw8zc1r4qjc0000gn/T/ipykernel_10871/1668609067.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks['centroid'] = blocks['geometry'].centroid


In [4]:
# Получение 40 случайных строк из исходного GeoDataFrame
blocks_40 = blocks.sample(n=40, random_state=1)  # Используйте другой random_state для другой выборки

# Создание нового GeoDataFrame на основе случайных строк
blocks_40 = gpd.GeoDataFrame(blocks_40, geometry='geometry', crs=blocks_40.crs)

In [5]:
blocks_40.reset_index(drop=True, inplace=True)
blocks_40['vrp_id'] = blocks_40.index
blocks_40.head()

,geometry,id,vrp_id
0,POINT (30.23725 59.94681),91,0
1,POINT (30.23616 59.93375),443,1
2,POINT (30.27768 59.94933),776,2
3,POINT (30.23291 59.95444),482,3
4,POINT (30.27220 59.93560),1073,4


In [6]:
# Функция для извлечения координат из геометрии
def extract_coordinates(row):
    return row['geometry'].x, row['geometry'].y

# Применение функции к каждой строке GeoDataFrame
blocks_40['coordinates'] = blocks_40.apply(extract_coordinates, axis=1)

# Преобразование координат в массив numpy
coordinates_array = np.array(blocks_40['coordinates'].tolist())

coordinates_array

array([[30.23725223, 59.94681097],
       [30.23615802, 59.93375286],
       [30.277682  , 59.94933284],
       [30.23291334, 59.95444208],
       [30.27220029, 59.9355989 ],
       [30.2920856 , 59.94028113],
       [30.24740838, 59.95465574],
       [30.2465466 , 59.94967585],
       [30.27443905, 59.93945694],
       [30.28112209, 59.94055325],
       [30.26709624, 59.93848723],
       [30.26132438, 59.93312537],
       [30.2253302 , 59.9326824 ],
       [30.20965571, 59.95089336],
       [30.24014745, 59.95016267],
       [30.27370056, 59.93642756],
       [30.23820194, 59.93056522],
       [30.22145889, 59.93357544],
       [30.26736085, 59.95338766],
       [30.28285029, 59.9481095 ],
       [30.2742119 , 59.93677489],
       [30.24658721, 59.93650089],
       [30.27102015, 59.93064191],
       [30.2679881 , 59.93489392],
       [30.25979894, 59.94808359],
       [30.23831776, 59.94709691],
       [30.26626537, 59.94770951],
       [30.22902667, 59.9585606 ],
       [30.22322408,

In [7]:
# Получение списка id кварталов из GeoDataFrame
quarters_ids = blocks_40['id'].tolist()

# Фильтрация матрицы связанности по кварталам из GeoDataFrame
filtered_matrix_df = accessibility_matrix.loc[quarters_ids, quarters_ids]

# Вывод отфильтрованной матрицы связанности
filtered_matrix_df.head()

id,91,443,776,482,1073,810,556,547,215,1093,...,767,744,1126,1118,680,201,1070,977,303,248
id,,,,,,,,,,,,,,,,,,,,,
91,0.0,6.1,11.8,7.9,12.4,15.9,9.5,7.7,12.6,12.7,...,9.6,14.6,14.8,14.6,12.3,12.4,10.3,9.2,9.8,14.8
443,5.0,0.0,13.0,9.9,10.1,15.7,11.5,9.7,12.4,12.5,...,10.8,13.6,14.7,13.6,10.0,13.6,11.5,3.6,11.0,15.4
776,12.2,15.0,0.0,14.5,7.9,6.0,11.5,9.9,5.5,5.6,...,2.1,7.3,4.9,4.9,7.8,6.2,5.2,15.1,7.4,4.3
482,5.3,10.2,14.5,0.0,16.3,19.8,7.5,5.7,16.5,16.6,...,13.5,18.5,18.7,18.5,16.2,11.4,10.9,13.6,9.8,18.7
1073,11.5,11.2,8.7,15.1,0.0,5.6,13.7,10.5,3.4,3.2,...,7.5,3.5,4.6,3.5,0.1,10.3,8.2,7.8,8.0,5.9


In [8]:
array_matrix = filtered_matrix_df.values.astype(int)
# array_result = array_result.reshape(-1, 1)
array_matrix

array([[ 0,  6, 11, ...,  9,  9, 14],
       [ 5,  0, 12, ...,  3, 10, 15],
       [12, 15,  0, ..., 15,  7,  4],
       ...,
       [ 8,  3, 14, ...,  0, 12, 14],
       [ 9, 12,  6, ..., 13,  0, 10],
       [14, 14,  4, ..., 13,  9,  0]])

In [9]:
demands


,date,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-01-01 01:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-01-01 02:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2022-01-01 03:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2022-01-01 04:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15307,2023-09-30 19:00:00,88,62,42,41,80,31,79,0,66,...,77,80,86,67,75,65,104,72,41,50
15308,2023-09-30 20:00:00,76,68,38,37,79,29,73,0,61,...,68,70,80,62,76,61,88,68,32,47
15309,2023-09-30 21:00:00,65,46,30,33,69,19,56,0,54,...,44,61,80,59,74,56,56,59,16,44
15310,2023-09-30 22:00:00,40,0,19,27,48,0,0,0,38,...,0,43,63,42,59,41,0,44,0,41


In [10]:
import pandas as pd

# Предположим, что ваш DataFrame называется orders_df
df = demands
# Преобразуем столбец 'date' в datetime
df['date'] = pd.to_datetime(df['date'])

# Устанавливаем 'date' в качестве индекса
df.set_index('date', inplace=True)

# Агрегируем данные по дням и суммируем заказы
daily_orders = df.resample('D').sum()

# Выводим новый датафрейм
daily_orders


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,653,0,0,0,510,175,498,597,385,182,...,574,436,806,568,811,547,705,0,0,0
2022-01-02,624,0,0,0,485,167,474,569,367,171,...,546,413,768,539,773,520,670,0,0,0
2022-01-03,343,0,0,0,267,94,262,312,202,94,...,303,229,423,297,426,286,370,0,0,0
2022-01-04,312,0,0,0,243,84,235,287,183,86,...,276,207,384,269,386,262,336,0,0,0
2022-01-05,342,0,0,0,266,91,262,312,202,95,...,301,228,423,295,424,285,370,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-26,465,289,240,249,407,153,407,0,333,337,...,379,484,502,357,417,318,556,414,200,299
2023-09-27,512,320,262,275,447,167,445,0,365,368,...,416,533,551,395,458,351,609,457,221,329
2023-09-28,558,347,286,298,488,181,484,0,399,402,...,454,582,601,427,504,383,665,497,240,356


In [11]:
# Функция, которая принимает дату и возвращает список заказов на эту дату
def get_orders_by_date(date, array):
    if array == True:
        try:
            orders = daily_orders.loc[date]
            return np.array(orders)
        except KeyError:
            return np.array([])  # Если дата не найдена, возвращаем пустой список
    else:
        try:
            orders = daily_orders.loc[date]
            return orders.tolist()
        except KeyError:
            return []  # Если дата не найдена, возвращаем пустой список
# Пример использованияЫ
input_date = '2023-03-19'  # Замените на нужную дату
result = get_orders_by_date(input_date, array = False)
print(f"Заказы на {input_date}: {result}")

Заказы на 2023-03-19: [881, 417, 554, 458, 669, 267, 734, 726, 602, 572, 806, 854, 648, 484, 617, 624, 707, 762, 255, 959, 829, 790, 696, 777, 1018, 979, 924, 575, 896, 920, 725, 773, 1052, 711, 832, 617, 1004, 614, 64, 164]


## OR-Tools

In [13]:
import networkx as nx
from cvrp_ortools_base import solve_cvrp
from collections import Counter


In [28]:
# Создание словаря
id_to_vrp_id_dict = dict(zip(blocks_40['id'].astype(str), blocks_40['vrp_id']))

# Вывод словаря
mapping = id_to_vrp_id_dict

# Назначение узла-депо
start_node = '91'
# Задать доступное количество транспортных средств (фиксированное)
num_vehicles = 3

depo_index = mapping[start_node]
# Для каждой даты решить задачу CVRP
num_vehicles = 3
demand = get_orders_by_date(input_date, array = False)

time_matrix = array_matrix

total_time = 0
vehicles_by_date = {}
# for date in dates:
#     print(date)
#     demand, demand_dict = calc_demand(G_demand, [date], mapping)
#     print('Cпрос на дату: ', dict(demand_dict))
#     print('')
#     # Запустить решение CVRP при заданных параметрах
vehicles_trips, covered_nodes, routing_time = solve_cvrp(time_matrix, demand, depo_index, num_vehicles, mapping)
# Определить, сколько машин было задействовано
vehicles = 0
for k in vehicles_trips.keys():
    if vehicles_trips[k]>0:
        vehicles+=1
# vehicles_by_date[date] = vehicles
# Накопить общее время машин в пути
total_time += routing_time
print('---------------------------------------------------------------------')
print('')

print('Выезжало машин:')
print(f'Общее время в пути: {round(total_time/60,2)} час.')

Route for vehicle 0:
Node '91' Delivered(881) -> Node '325' Delivered(1671) -> Node '686' Delivered(2477) -> Node '736' Delivered(3101) -> Node '680' Delivered(3933) -> Node '215' Delivered(4535) -> Node '1093' Delivered(5107) -> Node '744' Delivered(5880) -> Node '1118' Delivered(6591) -> Node '1126' Delivered(7643) -> Node '810' Delivered(7910) -> Node '248' Delivered(8074) -> Node '772' Delivered(9033) -> Node '776' Delivered(9587) -> Node '767' Delivered(10312) -> Node '201' Delivered(10929) -> Node '204' Delivered(11184) -> Node '556' Delivered(11918) -> Node '315' Delivered(12838) -> Node '546' Delivered(13455) -> Node '482' Delivered(13913) -> Node '60' Delivered(14488) -> Node '1342' Delivered(14972) ->  Node '91' Delivered(14972)
Time of the route: 98min
Load of the route: 14972

Route for vehicle 1:
Node '91' Delivered(881) ->  Node '91' Delivered(881)
Time of the route: 0min
Load of the route: 881

Route for vehicle 2:
Node '91' Delivered(881) -> Node '1183' Delivered(1860) 

## GNN

In [30]:
import torch
import os
import numpy as np
from torch_geometric.data import Data, DataLoader
from VRP.creat_vrp import reward1,creat_instance
from VRP.VRP_Actor import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

/Users/mvin/Code/DRL-and-graph-neural-network-for-routing-problems/vrptest/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
def discrete_cmap(N, base_cmap=None):
    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

def plot_vehicle_routes(data, route, ax1, markersize=5, visualize_demands=False, demand_scale=1, round_demand=False):

    plt.rc('font', family='Times New Roman', size=10)

    routes = [r[r != 0] for r in np.split(route.cpu().numpy(), np.where(route.cpu().numpy() == 0)[0]) if (r != 0).any()]
    print(routes)
    depot = data.x[0].cpu().numpy()
    locs = data.x[1:].cpu().numpy()
    demands = data.demand.cpu().numpy()
    demands=demands[1:]

    capacity = data.capcity

    x_dep, y_dep = depot
    ax1.plot(x_dep, y_dep, 'sk', markersize=markersize * 4)
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)

    legend = ax1.legend(loc='upper center')

    cmap = discrete_cmap(len(routes) + 2, 'nipy_spectral')
    dem_rects = []
    used_rects = []
    cap_rects = []
    qvs = []
    total_dist = 0
    for veh_number, r in enumerate(routes):
        color = cmap(len(routes) - veh_number)  # Invert to have in rainbow order

        route_demands = demands[r - 1]
        coords = locs[r - 1, :]
        xs, ys = coords.transpose()

        total_route_demand = sum(route_demands)
        #assert total_route_demand <= capacity
        if not visualize_demands:
            ax1.plot(xs, ys, 'o', mfc=color, markersize=markersize, markeredgewidth=0.0)

        dist = 0
        x_prev, y_prev = x_dep, y_dep
        cum_demand = 0
        for (x, y), d in zip(coords, route_demands):
            dist += np.sqrt((x - x_prev) ** 2 + (y - y_prev) ** 2)
            cap_rects.append(Rectangle((x, y), 0.01, 0.1))
            used_rects.append(Rectangle((x, y), 0.01, 0.1 * total_route_demand / capacity))
            dem_rects.append(Rectangle((x, y + 0.1 * cum_demand / capacity), 0.01, 0.1 * d / capacity))

            x_prev, y_prev = x, y
            cum_demand += d

        dist += np.sqrt((x_dep - x_prev) ** 2 + (y_dep - y_prev) ** 2)
        total_dist += dist
        qv = ax1.quiver(
            xs[:-1],
            ys[:-1],
            xs[1:] - xs[:-1],
            ys[1:] - ys[:-1],
            scale_units='xy',
            angles='xy',
            scale=1,
            color=color,
            label='R{}, N({}), C {} / {}, D {:.2f}'.format(
                veh_number,
                len(r),
                int(total_route_demand) if round_demand else total_route_demand,
                int(capacity) if round_demand else capacity,
                dist
            )
        )

        qvs.append(qv)

    ax1.set_title('Sampling1280,{} routes, total distance {:.2f}'.format(len(routes), total_dist), family='Times New Roman',size=20)

    ax1.legend(handles=qvs)
    plt.legend(loc=1)
    pc_cap = PatchCollection(cap_rects, facecolor='whitesmoke', alpha=1.0, edgecolor='lightgray')
    pc_used = PatchCollection(used_rects, facecolor='lightgray', alpha=1.0, edgecolor='lightgray')
    pc_dem = PatchCollection(dem_rects, facecolor='black', alpha=1.0, edgecolor='black')

    if visualize_demands:
        ax1.add_collection(pc_cap)
        ax1.add_collection(pc_used)
        ax1.add_collection(pc_dem)
    plt.show()
    #plt.savefig("./temp{}.png".format(54), dpi=600, bbox_inches='tight')

In [64]:
def cal_cost(routes):
    # Вычисление длины каждого маршрута
    total_length = 0.0
    for route in routes:
        route_length = sum(array_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
        # route_length += array_matrix[0][route[0]]
        # route_length += array_matrix[route[-1]][0]
        total_length += route_length
    return total_length

In [92]:
datas = []
n_nodes = 40
actor = 51
x=0 # выбираем набор данных

node_ = coordinates_array
input_date = '2023-03-19'  # Замените на нужную дату
demand_ = get_orders_by_date(input_date, array = True)

capcity_ = np.array([65000]) #настройка вместимости транспорта

node_ = node_.reshape(-1, n_nodes, 2)
# node_,demand_=node_.reshape(-1,n_nodes,2),demand_.reshape(-1,n_nodes)

data_size = node_.shape[0]

np_matrix = np.asarray(array_matrix)
np_matrix = np_matrix.reshape(-1, 1)

edges_index = []
for i in range(n_nodes):
    for j in range(n_nodes):
        edges_index.append([i, j])
edges_index = torch.LongTensor(edges_index)
edges_index = edges_index.transpose(dim0=0, dim1=1)

agent = Model(3, 128, 1, 16, conv_laysers=4).to(device)
agent.to(device)
folder = 'trained'
filepath = os.path.join(folder, '%s' % actor)
if os.path.exists(filepath):
        path1 = os.path.join(filepath, 'actor.pt')
        agent.load_state_dict(torch.load(path1, device))

datas_ = []
batch_size1 = 100  # sampling batch_size
for y in range(1000):
    data = Data(x=torch.from_numpy(node_[x]).float(), 
                edge_index=edges_index,
                edge_attr=torch.from_numpy(np_matrix).float(),
                demand=torch.tensor(demand_).unsqueeze(-1).float(),
                capcity=torch.tensor(capcity_).unsqueeze(-1).float())
    datas_.append(data)
dl = DataLoader(datas_, batch_size=batch_size1)

min_tour=[]
min_cost=10000
T=4#Temperature hyperparameters

for batch in dl:
    costs = []
    with torch.no_grad():
        batch.to(device)
        tour1, _ = agent(batch, n_nodes * 2, False, T)
        for tour in tour1:
            routes_1 = [r[r != 0] for r in np.split(tour.cpu().numpy(), np.where(tour.cpu().numpy() == 0)[0]) if (r != 0).any()]
            cost = cal_cost(routes_1)
            costs.append(cost)
        id = np.array(costs).argmin()
        m_cost=np.array(costs).min()
        if m_cost<min_cost:
            min_cost=m_cost
            min_tour=tour1[id]

tour=min_tour.unsqueeze(-2)
print('Problem:VRP''%s' % n_nodes,'/ Average distance:', min_cost)
for i, (data, tour) in enumerate(zip(dl, tour)):
    routes = [r[r != 0] for r in np.split(tour.cpu().numpy(), np.where(tour.cpu().numpy() == 0)[0]) if (r != 0).any()]
    print(routes)

ValueError: Expected parameter probs (Tensor of shape (100, 40)) of distribution Categorical(probs: torch.Size([100, 40])) to satisfy the constraint Simplex(), but found invalid values:
tensor([[0.3071, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5802, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3572, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.2748, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [75]:
depot = data.x[0].cpu().numpy()
locs = data.x[1:].cpu().numpy()
demands = data.demand.cpu().numpy()
demands=demands[1:]
total = 0
for veh_number, r in enumerate(routes):
    route_demands = demands[r - 1]
    coords = locs[r - 1, :]
    xs, ys = coords.transpose()
    total_route_demand = sum(route_demands)
    total += total_route_demand
    print(veh_number, total_route_demand, len(r))
print('Total demands''%s' % total)


0 [7385.] 8
1 [1752.] 2
2 [896.] 1
3 [7223.] 11
4 [1462.] 3
5 [617.] 1
6 [2723.] 4
7 [1216.] 2
8 [417.] 1
9 [669.] 1
10 [255.] 1
11 [575.] 1
12 [267.] 1
13 [484.] 1
14 [734.] 1
Total demands[26675.]


In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [ ]:
array_matrix

array([[ 0,  6, 11, ...,  9,  9, 14],
       [ 5,  0, 12, ...,  3, 10, 15],
       [12, 15,  0, ..., 15,  7,  4],
       ...,
       [ 8,  3, 14, ...,  0, 12, 14],
       [ 9, 12,  6, ..., 13,  0, 10],
       [14, 14,  4, ..., 13,  9,  0]])